In [ ]:
import pandas as pd

df_base = pd.read_excel("../fetched_data/형법.xlsx")

df_base

In [8]:
import pandas as pd
import json

def df_to_jsonl(df : pd.DataFrame, path : str, law_kind: str):
    if not path.endswith(".jsonl"):
        raise ValueError("a File name must ends with .jsonl")

    dataset = list()
    
    # frame = {
    #     "id" : 0,
    #     "query": {
    #         "title": "",
    #         "content": "",
    #     },
    # }

    frame = {
        "id" : 0,
        "title" : "",
        "content" : "",
    }

    for i, row in df.iterrows():
        # 삭제된 조항은 PASS
        if row["content"].strip().endswith("삭제"):
            continue
        # 조
        article = (
            f"제{row['article']}조"
            if row["article-branch"] == 1
            else f"제{row['article']}조의{row['article-branch']}"
        )
        # 항 
        paragraph = f" 제{row['paragraph']}항" if row["paragraph"] != 0 else ""
        # 호
        subparagraph = f" 제{row['subparagraph']}호" if row["subparagraph"] != 0 else ""

        str_id = f"{law_kind} {article}{paragraph}{subparagraph}"  # string
        data = frame.copy()
        # data['query']['title'] = str_id
        # data['query']['content'] = row['content']
        data['title'] = str_id
        data['content'] = row['content']
        dataset.append(data)

    for id, data in enumerate(dataset):
        data['id'] = id

    # JSONL 파일로 저장
    with open(path, 'w', encoding='utf-8') as f:
        for item in dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

### 편/장 제목 (preamble == 1) 은 제외

In [9]:
df_base = pd.read_excel("../fetched_data/형법.xlsx")

df = df_base.loc[df_base['preamble'] == 0]

df = df.drop('preamble', axis=1)

df_to_jsonl(df, "../data/형법.jsonl", law_kind="형법")

## GPT 문장 생성으로 query-pos 튜닝 데이터셋 구축

### Asynchronous API 요청

In [ ]:
from openai import AsyncOpenAI
from pydantic import BaseModel
import pandas as pd
import json
import os
from tqdm.asyncio import tqdm_asyncio
import asyncio

class Triplet(BaseModel):
    id : int
    title : str
    query: str
    pos: list[str]

def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    # df['title'] = df['query'].apply(lambda x : x['title'])
    # df['content'] = df['query'].apply(lambda x : x['content'])
    # df.drop('query', axis=1, inplace=True)

    df['title'] = df.apply(lambda x : x['title'])
    df['content'] = df.apply(lambda x : x['content'])

    return df

async def process_row_async(row, instructions, client):
    id = row['id']
    title = row['title']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    try:
        response = await client.responses.parse(
            model='gpt-4.1',
            instructions=instructions,
            input=[{"role": "user", "content": prompt}],
            text_format=Triplet,
        )
        return dict(response.output_parsed)
    except Exception as e:
        print(f"Error in row {title} : {e}")
        return {'id': id , 'title': title, 'query': query, 'pos': []}
    
async def generate_sentences_async(instrctions, client, dataset : pd.DataFrame, dir_path: str, concurrency: int = 6):
    sem = asyncio.Semaphore(concurrency) # API 동시 요청 수 제한

    async def sem_task(row):
        async with sem:
            return await process_row_async(row, instrctions, client)
        
    tasks = [sem_task(row) for _, row in dataset.iterrows()]
    results = []

    for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Processing"):
        result = await coro
        if result:
            results.append(result)

    os.makedirs(dir_path, exist_ok=True)
    path = os.path.join(dir_path, "relevant_incidents.jsonl")

    # 비동기 처리 결과값들 원래 쿼리 데이터 순서대로 정렬
    sorted_results = sorted(results, key=lambda x : x['id'])

    with open(path, 'w', encoding='utf-8') as f:
        for res in sorted_results:
            json.dump(res, f, ensure_ascii=False)
            f.write('\n')

In [4]:
from datasets import load_dataset
import nest_asyncio
import asyncio

client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

dataset = load_dataset("json", data_files="../data/queries-edited-filtered.jsonl")['train']
ds_df = dataset_to_df(dataset)

ds_df[:30]

,id,title,content
0,219,제87조,제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문...
1,220,제87조제1호,우두머리는 사형 무기징역 또는 무기금고에 처한다
2,221,제87조제2호,모의에 참여하거나 지휘하거나 그 밖의 중요한 임무에 종사한 자는 사형 무기 또는 5...
3,222,제87조제3호,부화수행(附和隨行)하거나 단순히 폭동에만 관여한 자는 5년 이하의 징역이나 금고에 처한다
4,223,제88조,제88조(내란목적의 살인) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나...
5,229,제91조제1호,헌법 또는 법률에 정한 절차에 의하지 아니하고 헌법 또는 법률의 기능을 소멸시키는 것
6,230,제91조제2호,헌법에 의하여 설치된 국가기관을 강압에 의하여 전복 또는 그 권능행사를 불가능하게 ...
7,231,제92조,제92조(외환유치) 외국과 통모하여 대한민국에 대하여 전단을 열게 하거나 외국인과 ...
8,232,제93조,제93조(여적) 적국과 합세하여 대한민국에 항적한 자는 사형에 처한다
9,234,제94조제1항,적국을 위하여 모병한 자는 사형 또는 무기징역에 처한다


In [ ]:
k = 5
with open("gpt_prompt.txt", "r", encoding="utf-8") as f:
    instructions = f.read().replace("#K#", str(k))

nest_asyncio.apply() # jupyter notebook 자체적인 running event loop 가 존재하므로 실행 중 루프 (주피터 노트북) 내 중첩된 루프를 허용

asyncio.run(generate_sentences_async(instructions, client, ds_df, dir_path="../data/gpt_output"))

Processing: 100%|██████████| 363/363 [06:37<00:00,  1.09s/it]


### prompt test

In [14]:
from datasets import load_dataset
import pandas as pd

def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    df['title'] = df['query'].apply(lambda x : x['title'])
    df['content'] = df['query'].apply(lambda x : x['content'])

    df.drop('query', axis=1, inplace=True)
    
    return df

def print_prompt_row(row):
    id = row['id']
    title = row['title']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    print(prompt)

dataset = load_dataset("json", data_files="../data/queries-edited-filtered.jsonl")['train']
ds_df = dataset_to_df(dataset)
ds_df = ds_df.loc[(ds_df['id'] >= 751) & (ds_df['id'] <= 761)]

ds_df

for i, data in ds_df.iterrows():
    # print(data)
    mock_process_row(data)

"id" : 751, "title" : 제366조, "query" : "제366조(재물손괴등) 타인의 재물 문서 또는 전자기록등 특수매체기록을 손괴 또는 은닉 기타 방법으로 기 효용을 해한 자는 3년이하의 징역 또는 700만원 이하의 벌금에 처한다"
"id" : 752, "title" : 제367조, "query" : "제367조(공익건조물파괴) 공익에 공하는 건조물을 파괴한 자는 10년 이하의 징역 또는 2천만원 이하의 벌금에 처한다"
"id" : 754, "title" : 제368조제1항, "query" : "재물손괴죄를 범하여 사람의 생명 또는 신체에 대하여 위험을 발생하게 한 때에는 1년 이상 10년 이하의 징역에 처한다"
"id" : 755, "title" : 제368조제2항, "query" : "재물손괴죄 또는 중손괴의 죄를 범하여 사람을 상해에 이르게 한 때에는 1년 이상의 유기징역에 처한다. 사망에 이르게 한 때에는 3년 이상의 유기징역에 처한다"
"id" : 757, "title" : 제369조제1항, "query" : "단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 재물손괴죄를 범한 때에는 5년 이하의 징역 또는 1천만원 이하의 벌금에 처한다"
"id" : 758, "title" : 제369조제2항, "query" : "단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 중손괴죄를 범한 때에는 1년 이상의 유기징역 또는 2천만원 이하의 벌금에 처한다"
"id" : 759, "title" : 제370조, "query" : "제370조(경계침범) 경계표를 손괴 이동 또는 제거하거나 기타 방법으로 토지의 경계를 인식 불능하게 한 자는 3년 이하의 징역 또는 500만원 이하의 벌금에 처한다"
"id" : 761, "title" : 제372조, "query" : "제372조(동력) 손괴의 죄에 관한 본장에서 관리할 수 있는 동력은 재물로 간주한다"


In [13]:
dataset = load_dataset("json", data_files="../data/queries-edited-filtered.jsonl")['train']
ds_df = dataset_to_df(dataset)
ds_df = ds_df.loc[(ds_df['id'] >= 751) & (ds_df['id'] <= 761)]

ds_df

,id,title,content
355,751,제366조,제366조(재물손괴등) 타인의 재물 문서 또는 전자기록등 특수매체기록을 손괴 또는 ...
356,752,제367조,제367조(공익건조물파괴) 공익에 공하는 건조물을 파괴한 자는 10년 이하의 징역 ...
357,754,제368조제1항,재물손괴죄를 범하여 사람의 생명 또는 신체에 대하여 위험을 발생하게 한 때에는 1년...
358,755,제368조제2항,재물손괴죄 또는 중손괴의 죄를 범하여 사람을 상해에 이르게 한 때에는 1년 이상의 ...
359,757,제369조제1항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 재물손괴죄를 범한 때에는 ...
360,758,제369조제2항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 중손괴죄를 범한 때에는 1...
361,759,제370조,제370조(경계침범) 경계표를 손괴 이동 또는 제거하거나 기타 방법으로 토지의 경계...
362,761,제372조,제372조(동력) 손괴의 죄에 관한 본장에서 관리할 수 있는 동력은 재물로 간주한다


### 기존 jsonl 파일 데이터 포맷 변경

In [7]:
from datasets import load_dataset
import os
import re

def trans_format(fname : str, law_kind: str):
    ds = load_dataset('json', data_files=fname)['train']
    new_ds = ds.map(lambda x : {"id" : x['id'], "title" : f"{law_kind} {re.sub(r'(?<!^)(?=제)', ' ', x['query']['title'])}", 'content' : x['query']['content']})
    new_ds = new_ds.remove_columns(['query'])
    new_fname = os.path.splitext(fname)[0] + "_2" + ".jsonl"
    new_ds.to_json(new_fname, force_ascii=False)

trans_format("../data/queries-edited.jsonl", "형법")
trans_format("../data/queries-edited-filtered.jsonl", "형법")

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from datasets import load_dataset

corpus = load_dataset("json", data_files=f"../data/KLAID_test_corpus.jsonl")[
    "train"
]

corpus[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'id': '0',
 'text': ['피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.']}

In [15]:
corpus_text = [text for sub in corpus["text"] for text in sub]

In [17]:
corpus['text']

[['피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.'],
 ['피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 가로막고 백미러를 잡아당기는 등 약 10분 동안 소란을 피워 피해자의 택시 영업업무를 방해하였다.'],
 ['피고인은 2014. 12. 23. 18:00경 서울 구로구 C에 있는 피해자 D이 운영하는 E 오류동점 휴대폰 매장 내에서, 그곳 보관함에 있던 피해자 소유의 시가 699,600원 상당의 갤럭시S5블랙 휴대폰 1대를 피고인이 입고 있던 상의 속에 집어넣은 후 위 매장에서 가지고 나오는 방법으로 절취한 것을 비롯하여, 위 일시경부터 2015. 6. 13.경까지 별지 범죄일람표 기재와 같이 총 49회에 걸쳐 피해자 소유의 시가 124,020,600원 상당의 휴대폰 136대를 상습으로 절취하였다.'],
 ['피고인은 2019. 2. 25. 12:10경 전주시 덕진구 B에 있는 C에서 피해자 D(39세)과 함께 일용직 일을 마친 후 점심을 먹으려 하던 중, 피해자로부터 점심값을 계산 해달라는 얘기를 듣고 화가 나 양손으로 피해자의 얼굴을 2회 때려 피해자에게 약 6주간의 치료가 필요한 기타 광대뼈 및 상악골의 골절 등의 상해를 가하였다.'],
 ['피고인은 2014. 7. 9. 13:50경 광주 서구 무진대로 957 씨엘병원 앞 버스정류장 부근에서 피해자 C